In [7]:
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt            # library for visualization
import random  
import re                                  # library for regular expression operations
import string                              # for string operations
import os
from nltk.corpus import stopwords          # module for stop words that come with NLTK1
from nltk.stem import PorterStemmer        #modeule for stemming
from nltk.stem import WordNetLemmatizer      # module for lemmatization
from nltk import word_tokenize
from nltk.corpus import words
nltk.download('wordnet')
from nltk.tag import pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
import pyhdb
from hdbcli import dbapi
from sqlalchemy import create_engine
import petl as etl

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/archana.priyadharsini/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/archana.priyadharsini/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/archana.priyadharsini/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/archana.priyadharsini/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# os.chdir(r"D:\TVSM Projects\Call no similarity")
# os.getcwd()

In [9]:
#connection to hana database
import pyhdb
hanaserver_bip='10.121.3.243'
port_bip=30015
userid_bip='HPR_DEVELOP'
password_bip='HPRdevelop123*'

dfs=('df1',)
tables=('VW_FT_RPT_CUST_COMP',)
df_data={}

In [10]:
def Hana_Connection(table_name,table_df):
 
    for table_df,table_name in zip(dfs,tables):
        #connection = pyhdb.connect(hanaserver_hpr, port_hpr,userid_hpr, password_hpr)
        connection = dbapi.connect(hanaserver_bip, port_bip,userid_bip, password_bip)
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM VW_FT_RPT_CUST_COMP WHERE CONVCALLDATE between (select TO_DATS(ADD_DAYS(max(CONVCALLDATE), -45)) from VW_FT_RPT_CUST_COMP) and (select max(CONVCALLDATE) from VW_FT_RPT_CUST_COMP)")
        col=cursor.description 
        df_data[table_df] = pd.DataFrame(cursor.fetchall())
        col_head=[]
        for i in range (len(col)):
            col_head.append(col[i][0])
        df_data[table_df].columns=col_head
    
    #Variant table modeification
    df_variant=df_data['df1']
    
    return df_variant
df_variant=Hana_Connection(tables,dfs)  


In [11]:
data=df_variant.copy()
print(data['CALL_YEAR'].unique())
print(data['CONVCALLDATE'].max())
print(data['CONVCALLDATE'].min())
data.shape

[2022]
2022-03-01
2022-01-15


(16616, 37)

In [12]:
# #df= pd.read_csv("Off_Aug_Oct_CallRegister.csv")
# dt=pd.read_excel(r"TechCalls_Dec2021.xlsx")
# dt

In [13]:
# data=dt[['Dealer Code','Call No','Call Type','Description','Resolution','Total Call Age(DD:HH:MM)','Call Status']]

In [14]:
train_dealer= data['DEALER_ID'].reset_index(drop=True)
test_dealer= data['DEALER_ID'].reset_index(drop=True)
test_dealer.shape

(16616,)

In [15]:
train=data
test=data

In [16]:
print(train.shape)
print(test.shape)

(16616, 37)
(16616, 37)


In [17]:
code_tr=train['CALL_NO'].reset_index(drop=True)
code_ts=test['CALL_NO'].reset_index(drop=True)

In [18]:
l=train.__len__()
print(l)
l_test=test.__len__()
print(l_test)

16616
16616


In [19]:
#dictionary={'xl100':'mop','3w':'Threewheeler','stopper':'stoppage'}
new_stop=['Dear','dear','vehicle',"don't",'sec','min','second','minute','period',"''",'hours','hrs','hour','hr','month','minutes','-1','s','nos','nos`s','s',"'s",'qty','-','jan-','feb-','mar-','apr-','may-','jun-','jul-','july-','aug-','sep-','oct-','nov-','dec-','jan','feb','mar','apr','may','jun','jul','july','aug','sep','oct','nov','dec',"'jan","'feb","'mar","'apr","'may","'jun","'jul","'july","'aug","'sep","'oct","'nov","'dec"]

In [20]:
def process(explain):
    if(type(explain) is not str):
        explanation1=explain.to_string()
    else:
        explanation1= explain.lower()
    #for word in explain: 
       # if(word in dictionary):
          #  word=dictionary[word]
    explanation2 = re.sub(r'^RT[\s]+', '', explanation1)
    explanation2 = re.sub(r'https?:\/\/.*[\r\n]*', '', explanation2)
    explanation2 = re.sub(r'\d\d\d', '', explanation2)
    explanation2 = re.sub(r'\d\d', '', explanation2)
    explanation2 = re.sub(r'\d', '', explanation2)
    explanation2 = re.sub(r"[\-|\.]", '', explanation2)
    explanation2 = re.sub(r"'" ," " , explanation2)
    explanation2 = re.sub(r"\-" ," " , explanation2)
    explanation2 = re.sub(r"/" ,' ', explanation2)
    explanation2 = re.sub(r"â€“" ,' ', explanation2)
    explanation2 = re.sub(r'ã¢â€â“' ,' ', explanation2)
    explanation2 = re.sub(r"[\....]"," " , explanation2)
    tokens = nltk.word_tokenize(explanation2)
    stopwords_english = stopwords.words('english') 
    stopwords_english.extend(new_stop)
    expalanation_clean= []
    
        
    for word in tokens:                        
        if (word not in stopwords_english and 
            word not in string.punctuation):  
            expalanation_clean.append(word)
            
      #stemmer = PorterStemmer() 
    lemmatizer = WordNetLemmatizer()
    expalanation_stem = [] 
    for word, tag in pos_tag(expalanation_clean):
        if tag.startswith("NN"):
            lemmatize_word=lemmatizer.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            lemmatize_word=lemmatizer.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            lemmatize_word=lemmatizer.lemmatize(word, pos='a')
        else:
            lemmatize_word=word
      #lemmatize_word =  lemmatizer.lemmatize(word,pos='a') 
      #expalanation_stem.append(stem_word) 
        expalanation_stem.append(lemmatize_word)    
    return expalanation_stem

print (process("DMS VEHICLE INVOICING ISSUE"))

['dm', 'invoice', 'issue']


In [21]:
train_desc=train['CALL_DESCRIPTION'].reset_index(drop=True)
test_desc=test['CALL_DESCRIPTION'].reset_index(drop=True)

In [22]:
#Saved_file
exp_list=[]
for i in range(l):
    exp_list.append(process(train_desc[i]))
for i in range(l):
    x=exp_list[i].__len__() 
    print(i,exp_list[i],x)

0 ['complaint', 'mdadbd', 'framw', 'number', 'create'] 5
1 ['sir', 'pls', 'map', 'customercustomer', 'name', 'rajeshwari', 'regn', 'tnbp', 'frame', 'mdcgmk', 'customer', 'id', 'mobile'] 13
2 ['complaint', 'customer', 'id', 'modify', 'object', 'reference', 'set', 'instance', 'object'] 9
3 ['sir', 'cant', 'create', 'job', 'card', 'frame', 'mdcglh', 'reg', 'klar'] 9
4 ['sir', 'cant', 'create', 'job', 'card', 'frame', 'mddell', 'please', 'help', 'create', 'customer', 'name', 'abhijith', 'regno', 'klw', 'mobno', 'thank', 'u'] 18
5 ['new', 'job', 'card', 'entry', 'error'] 5
6 ['sir', 'culd', 'create', 'grn', 'jagadamba', 'aps', 'please', 'sloved', 'itthere', 'part', 'help', 'many', 'option', 'drop', 'menu', 'like', 'part', 'help', 'grn', 'apsetc', 'please', 'solve'] 22
7 ['sir', 'medam', 'job', 'card', 'promise', 'delivery', 'time', 'open', 'thank', 'sir', 'medamdelar', 'code'] 12
8 ['frame', 'mdakmn', 'complaint', 'duplicate', 'pan', 'already', 'exist'] 7
9 ['job', 'card', 'job', 'card', 'd

In [23]:
#target file
project_list=[]
for i in range(l_test):
    project_list.append(process(test_desc[i]))
for i in range(l_test):
    x=project_list[i].__len__()
    print(i,project_list[i],x)  

0 ['complaint', 'mdadbd', 'framw', 'number', 'create'] 5
1 ['sir', 'pls', 'map', 'customercustomer', 'name', 'rajeshwari', 'regn', 'tnbp', 'frame', 'mdcgmk', 'customer', 'id', 'mobile'] 13
2 ['complaint', 'customer', 'id', 'modify', 'object', 'reference', 'set', 'instance', 'object'] 9
3 ['sir', 'cant', 'create', 'job', 'card', 'frame', 'mdcglh', 'reg', 'klar'] 9
4 ['sir', 'cant', 'create', 'job', 'card', 'frame', 'mddell', 'please', 'help', 'create', 'customer', 'name', 'abhijith', 'regno', 'klw', 'mobno', 'thank', 'u'] 18
5 ['new', 'job', 'card', 'entry', 'error'] 5
6 ['sir', 'culd', 'create', 'grn', 'jagadamba', 'aps', 'please', 'sloved', 'itthere', 'part', 'help', 'many', 'option', 'drop', 'menu', 'like', 'part', 'help', 'grn', 'apsetc', 'please', 'solve'] 22
7 ['sir', 'medam', 'job', 'card', 'promise', 'delivery', 'time', 'open', 'thank', 'sir', 'medamdelar', 'code'] 12
8 ['frame', 'mdakmn', 'complaint', 'duplicate', 'pan', 'already', 'exist'] 7
9 ['job', 'card', 'job', 'card', 'd

# JACCARD COEFFICIENT

In [24]:
matched=[]
within_dealer=[]
within_dealer_cnt=[]
outside_dealer=[]
outside_dealer_cnt=[]

def checking(saved_files, target_file):
    target_len= target_file.__len__()
    for j in range(target_len):
        matched_list=[]
        within_dealer_above_50=[]
        outside_dealer_above_50=[]
        f=0
        trn=exp_list[j]
        for i in range(l_test):
            saved=project_list[i]
            x=[]
            len_saved=saved.__len__()
            for word in trn :
                exist_count =saved.count(word)
                if(exist_count>0):
                    x.append(word)
            matched_list.append(x)
            #print(round(x.__len__()/j.__len__(),2))
            #print(x.__len__())
            if(trn.__len__()+len_saved)!=0:
                if (train_dealer[j]==test_dealer[i] and code_tr[j]!=code_ts[i] and (round((2*x.__len__())/(trn.__len__()+len_saved),2))>0.75):
                    comment_with_code_resembel=[]
                    comment_with_code_resembel=[code_tr[i],(round((2*x.__len__())/(trn.__len__()+len_saved),2))]
                    within_dealer_above_50.append(comment_with_code_resembel)
                elif(train_dealer[j]!=test_dealer[i] and (round((2*x.__len__())/(trn.__len__()+len_saved),2))>0.75):
                    #print('greater')
                    comment_with_code_resembel_1=[]
                    comment_with_code_resembel_1=[code_tr[i],(round((2*x.__len__())/(trn.__len__()+len_saved),2))]
                    outside_dealer_above_50.append(comment_with_code_resembel_1)
        
        within_dealer_above_50.sort(key=lambda x:x[1],reverse=True)  
        outside_dealer_above_50.sort(key=lambda x:x[1],reverse=True)
        within_dealer.append(within_dealer_above_50)
        outside_dealer.append(outside_dealer_above_50)
        within_dealer_cnt.append(within_dealer_above_50.__len__())
        outside_dealer_cnt.append(outside_dealer_above_50.__len__())

In [25]:
checking(exp_list, project_list)

In [26]:
new_column_1 = pd.DataFrame({'within_dealer': within_dealer})
new_column_1
target1 = test.merge(new_column_1, left_index = True, right_index = True)
target1.shape
new_column_2 = pd.DataFrame({'outside_dealer': outside_dealer})
new_column_2
target2= target1.merge(new_column_2, left_index = True, right_index = True)
target2.shape
new_column_3 = pd.DataFrame({'within_Dealer_cnt': within_dealer_cnt})
new_column_3.shape
target3= target2.merge(new_column_3, left_index = True, right_index = True)
target3.shape
new_column_4 = pd.DataFrame({'outside_Dealer_cnt': outside_dealer_cnt})
new_column_4.shape
target4= target3.merge(new_column_4, left_index = True, right_index = True)
target4.shape

# Final Result
target4.to_csv('output_TechCalls_2022.csv', index = False)
print(target4.shape)
print(target4.columns)

(16616, 41)
Index(['CALL_ID', 'CALL_NO', 'CALL_MONTH', 'CALL_YEAR', 'CALL_DAT',
       'DATE_CALL', 'CALL_LOGED_BY', 'CONVCALLDATE', 'REF_PROJECT_ID',
       'PROJECT_CODE', 'PROJECT_DESC', 'MODIFIEDON', 'OPENTIME',
       'CALL_PRIORITY_DESC', 'CALL_CAT_DESC', 'CALL_SUBCAT_DESC', 'DEALER_ID',
       'AREADESC', 'DEALER_NAME', 'ZONEDESC', 'CUST_CITY', 'RESCODE',
       'CALL_DESCRIPTION', 'CALL_STATUS_DESC', 'REOPENDATE', 'SITE_DESC',
       'RES_DESC', 'CLOSEDATETIME', 'CLOSETIME', 'CALLCATEGORY', 'CLOSE_TIM',
       'PENDINGDAYS', 'LEADTIME', 'DMSSPENDINGDAYS', 'HR_DIFF', 'HR_MIN_SEC',
       'DA_HR', 'within_dealer', 'outside_dealer', 'within_Dealer_cnt',
       'outside_Dealer_cnt'],
      dtype='object')


WRITE BACK TO HANA DB

In [70]:
target4.head(n=200)
target4['outside_Dealer_cnt'].max()

2316

In [28]:
# Index(['CALL_ID', 'CALL_NO', 'CALL_MONTH', 'CALL_YEAR', 'CALL_DAT',
#        'DATE_CALL', 'CALL_LOGED_BY', 'CONVCALLDATE', 'REF_PROJECT_ID',
#        'PROJECT_CODE', 'PROJECT_DESC', 'MODIFIEDON', 'OPENTIME',
#        'CALL_PRIORITY_DESC', 'CALL_CAT_DESC', 'CALL_SUBCAT_DESC', 'DEALER_ID',
#        'AREADESC', 'DEALER_NAME', 'ZONEDESC', 'CUST_CITY', 'RESCODE',
#        'CALL_DESCRIPTION', 'CALL_STATUS_DESC', 'REOPENDATE', 'SITE_DESC',
#        'RES_DESC', 'CLOSEDATETIME', 'CLOSETIME', 'CALLCATEGORY', 'CLOSE_TIM',
#        'PENDINGDAYS', 'LEADTIME', 'DMSSPENDINGDAYS', 'HR_DIFF', 'HR_MIN_SEC',
#        'DA_HR', 'within_dealer', 'outside_dealer', 'within_Dealer_cnt',
#        'outside_Dealer_cnt'],

In [71]:
# final_df=target[['CALL_NO','within_dealer_above_50','outside_dealer_above_50','within_Dealer_cnt','outside_dealer_cnt']].reset_index(drop=True)
# data.head()
final_df=target4[['CALL_NO','DEALER_ID','CALL_DESCRIPTION','RES_DESC','within_Dealer_cnt','outside_Dealer_cnt']].reset_index(drop=True)
# final_df=target4[['CALL_NO']].reset_index(drop=True)
# final_df=target4[['DEALER_ID']].reset_index(drop=True)
final_df['DEALER_ID']= final_df['DEALER_ID'].astype(float)
# final_df=target4[['CALL_DESCRIPTION']].reset_index(drop=True)
# final_df=target4[['CALL_NO','within_dealer']].reset_index(drop=True)
final_df['CALL_NO']=final_df['CALL_NO'].astype(float)
final_df['within_Dealer_cnt']=final_df['within_Dealer_cnt'].astype(float)
final_df['outside_Dealer_cnt']=final_df['outside_Dealer_cnt'].astype(float)
# final_df['within_dealer']= final_df['within_dealer'].astype(str)


# final_df['within_Dealer_cnt'].astype(int)
# final_df['outside_Dealer_cnt'].astype(int)
final_df['outside_Dealer_cnt'].max()
final_df.dtypes



CALL_NO               float64
DEALER_ID             float64
CALL_DESCRIPTION       object
RES_DESC               object
within_Dealer_cnt     float64
outside_Dealer_cnt    float64
dtype: object

In [30]:
# target4=pd.read_csv('/Users/archana.priyadharsini/Desktop/Downloads/output_TechCalls_JanFeb2022.csv')
# target4.columns

In [72]:
hanaserver_bip='10.121.3.243'
port_bip=30015
userid_bip='HPR_DEVELOP'
password_bip='HPRdevelop123*'

In [74]:
def Hana_updation():
            connection= dbapi.connect(hanaserver_bip, port_bip, userid_bip, password_bip)
            print('a')
            cursor1 = connection.cursor()
            print('b')
            cursor1.execute('DROP TABLE CALL_CENTRE')
            create_query ='CREATE TABLE CALL_CENTRE ("CALL_NO" INT primary key,"DEALER_ID" INT,"CALL_DESCRIPTION" VARCHAR(2000),"RES_DESC" VARCHAR(2000),"WITHINDEALER_CNT" INT,"OUTSIDEDEALER_CNT" INT)' #should be same as the input table
            cursor1.execute(create_query)
            print('c')
            cols = ",".join([str(i) for i in final_df.columns.tolist()])
            print('d')
           
            for i, row in final_df.iterrows(): 
                ## SQL Server
                sql = "INSERT INTO CALL_CENTRE VALUES (" + "?,"*(len(row)-1) + "?);"

                cursor1.execute(sql, tuple(row.values))
                connection.commit()
              
cc=Hana_updation()


a
b
c
d


In [75]:
#connection to hana database
import pyhdb
hanaserver_bip='10.121.3.243'
port_bip=30015
userid_bip='HPR_DEVELOP'
password_bip='HPRdevelop123*'

dfs=('df1',)
tables=('VW_FT_RPT_CUST_COMP',)
df_data={}

def Hana_Connection(table_name,table_df):
 
    for table_df,table_name in zip(dfs,tables):
        #connection = pyhdb.connect(hanaserver_hpr, port_hpr,userid_hpr, password_hpr)
        connection = dbapi.connect(hanaserver_bip, port_bip,userid_bip, password_bip)
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM CALL_CENTRE")
        col=cursor.description 
        df_data[table_df] = pd.DataFrame(cursor.fetchall())
        col_head=[]
        for i in range (len(col)):
            col_head.append(col[i][0])
        df_data[table_df].columns=col_head
    
    #Variant table modeification
    cc=df_data['df1']
    
    return cc
cc=Hana_Connection(tables,dfs)  
cc.to_csv("output_TechCalls_2022.csv")
# cc['within_dealer'].unique()

In [ ]:

cc.to_csv("output_TechCalls_2022.csv")

In [67]:
cc['outside_Dealer_cnt'].max()

2316

In [ ]:
# #pip install wordcloud
# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator 

In [ ]:
# text = target.Description[0]
# wordcloud = WordCloud().generate(text)
# wordcloud

# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.show()

In [ ]:
#df["Dealer Code"]=df[["Dealer Code"]:2]

In [ ]:
#.........................................END.................................................................#

In [ ]:
# trn1=process("dear sir, job card closed but it's showing job completed status.job card details given below. job card no : 11017 job card date : 28/12/2019 invoice no : 7753 please do the needful.")
# target1=process("DEAR SIR; JOB CARD CANCELLED BUT IT'S SHOWING JOB COMPLETE STATUS: JOB CARD GIVEN BELOW JOB CARD NO:26233 JOB CARD DATE:21.10.2021")

# x=[]

# for word in trn1 :
#     exist_count =target1.count(word)
#     if(exist_count>0):
#         x.append(word)

# print(x.__len__(),trn1.__len__(),target1.__len__())
# print(round((2*x.__len__())/(trn1.__len__()+target1.__len__()),2))

#(round((2*x.__len__())/(trn.__len__()+len_saved),2))